In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 使用Dask进行分布式XGBoost训练

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/training/xgboost_data_parallel_training_on_cpu_using_dask.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> 在Colab中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fofficial%2Ftraining%2Fxgboost_data_parallel_training_on_cpu_using_dask.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"><br> 在Colab Enterprise中打开
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/training/xgboost_data_parallel_training_on_cpu_using_dask.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> 在Workbench中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/training/xgboost_data_parallel_training_on_cpu_using_dask.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> 在GitHub上查看
    </a>
  </td>
</table>

## 概述

本教程向您展示如何在Vertex AI上使用Dask创建一个可以处理大量训练数据的分布式自定义训练作业，使用XGBoost。

了解更多关于[自定义训练](https://cloud.google.com/vertex-ai/docs/training/custom-training)。

### 目标

在本教程中，您将学习如何使用Dask创建一个使用XGBoost的分布式训练作业。您将构建一个具有简单Dask配置的自定义docker容器，以运行自定义训练作业。当您的训练作业正在运行时，您可以访问Dask仪表板来监视集群、资源和计算的实时状态。

本教程使用以下Google Cloud ML服务：

- `Vertex AI Training`
- `Artifact Registry`

执行的步骤包括：

- 为您的Google Cloud项目配置`PROJECT_ID`和`LOCATION`变量。
- 创建一个Cloud Storage存储桶来存储您的模型工件。
- 构建一个承载您的训练代码的自定义Docker容器，并将容器镜像推送到Artifact Registry。
- 运行Vertex AI SDK CustomContainerTrainingJob。

### 数据集

本教程使用 <a href="https://scikit-learn.org/stable/auto_examples/datasets/plot_iris_dataset.html">IRIS 数据集</a>，该数据集预测鸢尾花的品种。

成本

本教程使用 Google Cloud 的计费组件：

* Vertex AI

* Cloud Storage

* Artifact Registry

了解 [Vertex AI 定价](https://cloud.google.com/vertex-ai/pricing)，[Cloud Storage 定价](https://cloud.google.com/storage/pricing)和[Artifact Registry](https://cloud.google.com/artifact-registry/pricing)，并使用[Pricing Calculator](https://cloud.google.com/products/calculator/)，
根据您的预期使用情况生成成本估算。

开始。

### 为Python安装Vertex AI SDK和其他必需的软件包

### 安装Vertex AI SDK for Python和其他必需的软件包

In [ ]:
! pip3 install --upgrade --quiet google-cloud-aiplatform

### 重新启动运行时（仅适用于Colab）

要使用新安装的软件包，您必须重新启动Google Colab上的运行时。

In [ ]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning"> 
<b>⚠️ 内核将重新启动。请等待它完成后再继续下一步。 ⚠️</b>
</div>

### 验证您的笔记本环境（仅适用于Colab）

在Google Colab上验证您的环境。

In [ ]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

### 设置Google Cloud项目信息并为Python初始化Vertex AI SDK

要开始使用Vertex AI，您必须拥有现有的Google Cloud项目并 [启用Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)。了解更多关于 [设置项目和开发环境](https://cloud.google.com/vertex-ai/docs/start/cloud-environment) 的信息。

In [ ]:
PROJECT_ID = "vertexai-service-project"  # "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

创建一个云存储存储桶

创建一个存储桶来存储中间产物，如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique-nm21"  # @param {type:"string"}

如果您的存储桶不存在：运行以下单元格以创建您的云存储桶。

In [ ]:
! gsutil mb -l $LOCATION -p $PROJECT_ID $BUCKET_URI

初始化Python的Vertex AI SDK

In [ ]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_URI, location=LOCATION)

**注意:** 为服务账户分配`Logs Writer`角色。

导入库并定义常量

In [ ]:
import os

# 创建一个自定义的Python训练包

在进行本地训练之前，您必须创建一个训练脚本文件和一个docker文件。

为您所有的训练代码创建一个`trainer`目录。

In [ ]:
PYTHON_PACKAGE_APPLICATION_DIR = "trainer"
!mkdir -p $PYTHON_PACKAGE_APPLICATION_DIR

###写培训脚本

`train.py`文件检查当前节点是主节点还是工作节点，并为主节点运行`dask-scheduler`，为工作节点运行`dask-worker`。工作节点通过`CLUSTER_SPEC`中指定的IP地址和端口号连接到主节点。

Dask调度程序设置并连接到工作节点后，调用`xgb.dask.train`通过Dask训练模型。一旦模型训练完成，模型将被上传到`AIP_MODEL_DIR`。

In [ ]:
%%writefile trainer/train.py
from dask.distributed import Client, wait
from xgboost.dask import DaskDMatrix
from google.cloud import storage
import xgboost as xgb
import dask.dataframe as dd
import sys
import os
import subprocess
import time
import json

IRIS_DATA_FILENAME = 'gs://cloud-samples-data/ai-platform/iris/iris_data.csv'
IRIS_TARGET_FILENAME = 'gs://cloud-samples-data/ai-platform/iris/iris_target.csv'
MODEL_FILE = 'model.bst'
MODEL_DIR = os.getenv("AIP_MODEL_DIR")
XGB_PARAMS = {
    'verbosity': 2,
    'learning_rate': 0.1,
    'max_depth': 8,
    'objective': 'reg:squarederror',
    'subsample': 0.6,
    'gamma': 1,
    'verbose_eval': True,
    'tree_method': 'hist',
    'nthread': 1
}


def square(x):
    return x ** 2

def neg(x):
    return -x

def launch(cmd):
    """ launch dask workers
    """
    return subprocess.check_call(cmd, stdout=sys.stdout, stderr=sys.stderr, shell=True)


def get_chief_ip(cluster_config_dict):
    if 'workerpool0' in cluster_config_dict['cluster']:
      ip_address = cluster_config_dict['cluster']['workerpool0'][0].split(":")[0]
    else:
      # if the job is not distributed, 'chief' will be populated instead of
      # workerpool0.
      ip_address = cluster_config_dict['cluster']['chief'][0].split(":")[0]

    print('The ip address of workerpool 0 is : {}'.format(ip_address))
    return ip_address

def get_chief_port(cluster_config_dict):

    if "open_ports" in cluster_config_dict:
      port = cluster_config_dict['open_ports'][0]
    else:
      # Use any port for the non-distributed job.
      port = 7777
    print("The open port is: {}".format(port))

    return port

if __name__ == '__main__':
    cluster_config_str = os.environ.get('CLUSTER_SPEC')
    cluster_config_dict  = json.loads(cluster_config_str)
    print(json.dumps(cluster_config_dict, indent=2))
    print('The workerpool type is:', flush=True)
    print(cluster_config_dict['task']['type'], flush=True)
    workerpool_type = cluster_config_dict['task']['type']
    chief_ip = get_chief_ip(cluster_config_dict)
    chief_port = get_chief_port(cluster_config_dict)
    chief_address = "{}:{}".format(chief_ip, chief_port)

    if workerpool_type == "workerpool0":
      print('Running the dask scheduler.', flush=True)
      proc_scheduler = launch('dask-scheduler --dashboard --dashboard-address 8888 --port {} &'.format(chief_port))
      print('Done the dask scheduler.', flush=True)

      client = Client(chief_address, timeout=1200)
      print('Waiting the scheduler to be connected.', flush=True)
      client.wait_for_workers(1)

      X = dd.read_csv(IRIS_DATA_FILENAME, header=None)
      y = dd.read_csv(IRIS_TARGET_FILENAME, header=None)
      X.persist()
      y.persist()
      wait(X)
      wait(y)
      dtrain = DaskDMatrix(client, X, y)

      output = xgb.dask.train(client, XGB_PARAMS, dtrain,  num_boost_round=100, evals=[(dtrain, 'train')])
      print("Output: {}".format(output), flush=True)
      print("Saving file to: {}".format(MODEL_FILE), flush=True)
      output['booster'].save_model(MODEL_FILE)
      bucket_name = MODEL_DIR.replace("gs://", "").split("/", 1)[0]
      folder = MODEL_DIR.replace("gs://", "").split("/", 1)[1]
      bucket = storage.Client().bucket(bucket_name)
      print("Uploading file to: {}/{}{}".format(bucket_name, folder, MODEL_FILE), flush=True)
      blob = bucket.blob('{}{}'.format(folder, MODEL_FILE))
      blob.upload_from_filename(MODEL_FILE)
      print("Saved file to: {}/{}".format(MODEL_DIR, MODEL_FILE), flush=True)

      # Waiting 10 mins to connect the Dask dashboard
      time.sleep(60 * 10)
      client.shutdown()

    else:
      print('Running the dask worker.', flush=True)
      client = Client(chief_address, timeout=1200)
      print('client: {}.'.format(client), flush=True)
      launch('dask-worker {}'.format(chief_address))
      print('Done with the dask worker.', flush=True)

      # Waiting 10 mins to connect the Dask dashboard
      time.sleep(60 * 10)


### 编写 Docker 文件
Docker文件用于构建自定义训练容器，并传递给Vertex Training。

In [ ]:
%%writefile Dockerfile
FROM us-docker.pkg.dev/vertex-ai/training/tf-cpu.2-9:latest
WORKDIR /root

# Install sudo
RUN apt-get update && apt-get -y install sudo

# Update the keyring in order to run apt-get update.
RUN rm -rf /usr/share/keyrings/cloud.google.gpg
RUN rm -rf /etc/apt/sources.list.d/google-cloud-sdk.list
RUN curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
RUN echo "deb https://packages.cloud.google.com/apt cloud-sdk main" | sudo tee -a /etc/apt/sources.list.d/google-cloud-sdk.list

# Install packages (without sudo)
RUN apt-get update && apt-get install -y telnet netcat iputils-ping net-tools

# Determine the default Python version
RUN echo python3 --version

# Install Python packages using the identified version
RUN python3 -m pip install 'xgboost>=1.4.2' 'dask-ml[complete]==2022.5.27' 'dask[complete]==2022.01.0' --upgrade
RUN python3 -m pip install dask==2022.01.0 distributed==2022.01.0 bokeh==2.4.3 dask-cuda==22.2.0 click==8.0.1 --upgrade
RUN python3 -m pip install gcsfs --upgrade


# Make sure gsutil will use the default service account
RUN echo '[GoogleCompute]\nservice_account = default' > /etc/boto.cfg

# Copies the trainer code
RUN mkdir /root/trainer
COPY trainer/train.py /root/trainer/train.py

# Sets up the entry point to invoke the trainer.
ENTRYPOINT ["python3", "trainer/train.py"]


## 创建一个自定义训练作业

### 构建一个自定义训练容器

#### 启用 Artifact Registry API
您必须为您的项目启用Artifact Registry API。您将把自定义训练容器存储在Artifact Registry中。

<a href="https://cloud.google.com/artifact-registry/docs/enable-service">了解更多关于启用服务</a>。

In [ ]:
! gcloud services enable artifactregistry.googleapis.com

### 创建一个私有的Docker仓库
您的第一步是在Artifact Registry中创建一个Docker仓库。

1. 运行`gcloud artifacts repositories create`命令来创建一个新的Docker仓库，使用您的位置和描述为`docker repository`。

2. 运行`gcloud artifacts repositories list`命令来验证您的仓库已经创建。

In [ ]:
PRIVATE_REPO = "my-docker-repo"

if os.getenv("IS_TESTING"):
    ! sudo apt-get update --yes && sudo apt-get --only-upgrade --yes install google-cloud-sdk-cloud-run-proxy google-cloud-sdk-harbourbridge google-cloud-sdk-cbt google-cloud-sdk-gke-gcloud-auth-plugin google-cloud-sdk-kpt google-cloud-sdk-local-extract google-cloud-sdk-minikube google-cloud-sdk-app-engine-java google-cloud-sdk-app-engine-go google-cloud-sdk-app-engine-python google-cloud-sdk-spanner-emulator google-cloud-sdk-bigtable-emulator google-cloud-sdk-nomos google-cloud-sdk-package-go-module google-cloud-sdk-firestore-emulator kubectl google-cloud-sdk-datastore-emulator google-cloud-sdk-app-engine-python-extras google-cloud-sdk-cloud-build-local google-cloud-sdk-kubectl-oidc google-cloud-sdk-anthos-auth google-cloud-sdk-app-engine-grpc google-cloud-sdk-pubsub-emulator google-cloud-sdk-datalab google-cloud-sdk-skaffold google-cloud-sdk google-cloud-sdk-terraform-tools google-cloud-sdk-config-connector
    ! gcloud components update --quiet

! gcloud artifacts repositories create {PRIVATE_REPO} --repository-format=docker --location={LOCATION} --description="Docker repository"

! gcloud artifacts repositories list

In [ ]:
TRAIN_IMAGE = (
    f"{LOCATION}-docker.pkg.dev/"
    + PROJECT_ID
    + f"/{PRIVATE_REPO}"
    + "/dask_support:latest"
)
print("Deployment:", TRAIN_IMAGE)

将Docker验证到您的仓库
配置认证到您的私人仓库
在您可以将容器镜像推送到或从您的Artifact Registry仓库拉取之前，您必须配置Docker使用gcloud命令行工具来认证对您位置的Artifact Registry的请求。在Colab上，您将不得不使用Cloud Build作为docker命令不可用。

In [ ]:
import sys

IS_COLAB = "google.colab" in sys.modules
if not IS_COLAB:
    ! gcloud auth configure-docker {LOCATION}-docker.pkg.dev --quiet

设置自定义的Docker容器镜像。

1. 从Docker Hub拉取对应的CPU或GPU Docker镜像。
2. 为镜像创建标签以在Artifact Registry中注册。
3. 将镜像注册到Artifact Registry。

In [ ]:
if not IS_COLAB:
    ! docker build -t $TRAIN_IMAGE -f Dockerfile .
    ! docker push $TRAIN_IMAGE

## 使用Cloud Build构建和推送自定义docker容器镜像

使用Cloud Build构建并推送一个Docker镜像

In [ ]:
if IS_COLAB:
    !  gcloud builds submit --timeout=1800s --region={LOCATION} --tag $TRAIN_IMAGE

使用SDK（选项1）或gcloud（选项2）运行培训作业。

运行Vertex AI SDK CustomContainerTrainingJob

您可以指定字段enable_web_access和enable_dashboard_access。 enable_web_access可启用作业的交互式 shell，而enable_dashboard_access允许访问 dask 仪表板。

In [ ]:
gcs_output_uri_prefix = f"{BUCKET_URI}/output"
replica_count = 2
machine_type = "n1-standard-4"
display_name = "test_display_name"
DEPLOY_IMAGE = "us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-8:latest"

custom_container_training_job = aiplatform.CustomContainerTrainingJob(
    display_name=display_name,
    model_serving_container_image_uri=DEPLOY_IMAGE,
    container_uri=TRAIN_IMAGE,
)

custom_container_training_job.run(
    base_output_dir=gcs_output_uri_prefix,
    replica_count=replica_count,
    machine_type=machine_type,
    enable_dashboard_access=True,
    enable_web_access=True,
    sync=False,
)

等几分钟，定制工作即将开始。

In [ ]:
import time

time.sleep(60 * 10)

In [ ]:
try:
    print(f"Custom Training Job Name: {custom_container_training_job.resource_name}")
    print(f"GCS Output URI Prefix: {gcs_output_uri_prefix}")
except Exception as e:
    print(e)

您可以在云控制台界面中访问自定义作业的链接。

In [ ]:
try:
    print(
        f"Custom Training Job URI: {custom_container_training_job._custom_job_console_uri()}"
    )
except Exception as e:
    print(e)

一旦工作进入状态“运行中”，您可以在这里访问Web访问和仪表板访问的URI:

In [ ]:
try:
    print(
        f"Web Access and Dashboard URIs: {custom_container_training_job.web_access_uris}"
    )
except Exception as e:
    print(e)

交互式 shell 具有格式为 "workerpool0-0" 的密钥，而仪表板 URI 具有格式为 "workerpool0-0:" + 端口号（例如，在本示例中为 workerpool0-0:8888）的密钥。在 Cloud Console UI 中的自定义作业页面上，您还可以为 web 访问的 "workerpool0-0" 启动 web 终端，或者单击 "启动 web 终端" 为了仪表板访问的 "workerpool0-0:" + 端口号。

请注意，您只能在作业运行时访问交互式 shell 和仪表板。如果您在 UI 中看不到 "启动 web 终端" 或者在 Web 访问和仪表板 URI 命令输出中看不到 URI，这可能是因为 Vertex AI 尚未开始运行您的作业，或作业已经完成或失败。如果作业状态为排队或挂起，请等待一分钟；然后尝试刷新页面，或再次尝试命令。

### 2. 使用gcloud运行CustomContainerTraining作业

您也可以使用gcloud命令创建一个训练作业。 使用gcloud命令，您可以指定字段enableWebAccess和enableDashboardAccess。 enableWebAccess启用作业的交互式shell，enableDashboardAccess允许访问Dask仪表板。

In [ ]:
%%bash -s "$BUCKET_URI/output" "$TRAIN_IMAGE"

cat <<EOF >config.yaml
enableDashboardAccess: true
enableWebAccess: true
# Creates two worker pool. The first worker pool is a chief and the second is
# a worker.
workerPoolSpecs:
  - machineSpec:
      machineType: n1-standard-8
    replicaCount: 1
    containerSpec:
      imageUri: $2
  - machineSpec:
      machineType: n1-standard-8
    replicaCount: 1
    containerSpec:
      imageUri: $2
baseOutputDirectory:
  outputUriPrefix: $1
EOF
cat config.yaml

以下命令创建一个训练工作。

In [ ]:
! gcloud ai custom-jobs create --region=us-central1 --config=config.yaml --display-name={display_name}

等待几分钟，让定制工作开始。

In [ ]:
import time

time.sleep(60 * 10)

#### 访问gcloud自定义作业的仪表板和交互式shell

作业创建后，您可以使用`gcloud ai custom-jobs describe`命令来打印webAccessUris字段，从而访问Web访问URI和仪表板访问URI。交互式shell的密钥格式为"workerpool0-0"，而仪表板URI的密钥格式为"workerpool0-0:" + 端口号（在本例中为workerpool0-0:8888）。

您也可以在Cloud控制台UI中找到这些链接。在Cloud控制台UI中，转到Vertex AI部分，然后进入训练和自定义作业。单击您自定义训练作业的名称。在作业页面上，单击"启动Web终端"以获取Web访问的"workerpool0-0"，或者单击"启动Web终端"以获取仪表板访问的"workerpool0-0:" + 端口号。

请注意，只有在作业运行时才能访问交互式shell和仪表板。如果您在UI中看不到"启动Web终端"或在gcloud命令输出中看不到URI，可能是因为Vertex AI尚未开始运行您的作业，或者作业已经完成或失败。如果作业状态为排队或挂起，请等待一分钟，然后尝试刷新页面或重新运行gcloud命令。

#### 故障排除

交互式 shell 可以用于调试对 dask 仪表板的访问。您可以通过以下命令获取仪表板地址。

In [ ]:
# Note the following command should run inside the interactive shell.
# printenv | grep AIP_DASHBOARD_PORT

然后您可以检查是否有仪表板监控该端口。

In [ ]:
# Note the following command should run inside the interactive shell.
# netstat -ntlp

您可以手动调高仪表盘实例。

In [ ]:
# Note the following command should run inside the interactive shell.
# dask-scheduler --dashboard-address :port_number

查看培训输出物品

In [ ]:
! gsutil ls $gcs_output_uri_prefix/model/

清理

要清理此项目中使用的所有Google Cloud资源，您可以删除用于本教程的[Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源：

- Cloud Storage存储桶
- Cloud Vertex训练作业

In [ ]:
import logging
import traceback

# Set this to true only if you'd like to delete your bucket
delete_bucket = False
delete_application_directory = False

! gsutil rm -rf $gcs_output_uri_prefix

if delete_bucket:
    ! gsutil rm -r $BUCKET_URI

try:
    custom_container_training_job.delete()
except Exception as e:
    logging.error(traceback.format_exc())
    print(e)

# Delete application directory
if delete_application_directory:
    ! rm -rf trainer config.yaml Dockerfile

! gcloud artifacts repositories delete {PRIVATE_REPO} --location={LOCATION} --quiet